D'acord, l'objectiu és poder determinar quin dels dos equips guanyarà la partida

In [48]:
# Importar llibreries necessàries
import pandas as pd
import numpy as np
import sklearn as sk

In [49]:
DataSet = pd.read_csv("../LOL - Dataset/games.csv")

DataSet.head(10)

,gameId,creationTime,gameDuration,seasonId,winner,firstBlood,firstTower,firstInhibitor,firstBaron,firstDragon,...,t2_towerKills,t2_inhibitorKills,t2_baronKills,t2_dragonKills,t2_riftHeraldKills,t2_ban1,t2_ban2,t2_ban3,t2_ban4,t2_ban5
0,3326086514,1504279457970,1949,9,1,2,1,1,1,1,...,5,0,0,1,1,114,67,43,16,51
1,3229566029,1497848803862,1851,9,1,1,1,1,0,1,...,2,0,0,0,0,11,67,238,51,420
2,3327363504,1504360103310,1493,9,1,2,1,1,1,2,...,2,0,0,1,0,157,238,121,57,28
3,3326856598,1504348503996,1758,9,1,1,1,1,1,1,...,0,0,0,0,0,164,18,141,40,51
4,3330080762,1504554410899,2094,9,1,2,1,1,1,1,...,3,0,0,1,0,86,11,201,122,18
5,3287435705,1501667992132,2059,9,1,2,2,1,1,2,...,6,0,0,3,0,119,134,154,63,31
6,3314215542,1503430065724,1993,9,1,1,2,1,1,1,...,2,0,0,0,0,75,42,31,40,429
7,3329224025,1504472363684,1334,9,1,1,1,0,0,2,...,0,0,0,2,0,157,38,122,40,238
8,3318040883,1503686577549,1387,9,2,2,2,2,0,2,...,8,1,0,2,1,28,51,53,157,40
9,3327786881,1504385918871,2681,9,2,2,2,2,2,2,...,8,3,1,2,0,64,141,84,67,53


In [50]:
# Mirar el percentatge  i nombre de valors nuls per columna
null_percentage = DataSet.isnull().mean() * 100
null_count = DataSet.isnull().sum()

print("Percentatge de valors nuls per columna:")
print(null_percentage)
print("Nombre de valors nuls per columna:")
print(null_count)

Percentatge de valors nuls per columna:
gameId          0.0
creationTime    0.0
gameDuration    0.0
seasonId        0.0
winner          0.0
               ... 
t2_ban1         0.0
t2_ban2         0.0
t2_ban3         0.0
t2_ban4         0.0
t2_ban5         0.0
Length: 61, dtype: float64
Nombre de valors nuls per columna:
gameId          0
creationTime    0
gameDuration    0
seasonId        0
winner          0
               ..
t2_ban1         0
t2_ban2         0
t2_ban3         0
t2_ban4         0
t2_ban5         0
Length: 61, dtype: int64


### Tractament de NaNs
Sorprenentment, podem observar que cap de les columnes tenen nans. Això comporta que no hi ha falta el tractament de NaNs.


### Tipus de Dades
Com sabem primer em de saber els tipus de dades que tenim en cadascuna de les columnes, en cas de tenir alguna variable categòrica podem fer encoding per poder tractar-les.

In [51]:
# Mirar els tipus de dades de cada columna
data_types = DataSet.dtypes
print("Tipus de dades per columna:")
print(data_types)

Tipus de dades per columna:
gameId          int64
creationTime    int64
gameDuration    int64
seasonId        int64
winner          int64
                ...  
t2_ban1         int64
t2_ban2         int64
t2_ban3         int64
t2_ban4         int64
t2_ban5         int64
Length: 61, dtype: object


Totes les columnes són de tipus númeric.

El següent pas és mirar si hi ha dades completament duplicades i en cas que hi hagin eliminar-les.


In [52]:
# Primer mirem quines són totalment identiques
duplicated_rows = DataSet[DataSet.duplicated(keep=False)]
num_duplicated_rows = len(duplicated_rows)
print(f"Nombre de files duplicades: {num_duplicated_rows} de {len(DataSet)}")

# Si n'hi ha, les eliminem
DataSet_cleaned = DataSet.drop_duplicates()
print(f"Nombre de files després d'eliminar duplicades: {len(DataSet_cleaned)}")

Nombre de files duplicades: 861 de 51490
Nombre de files després d'eliminar duplicades: 51053


### Anàlisi de cadascuna de les columnes individualment

In [53]:
def analyze_numeric_column(df, col, target='Survived', show_plots=True, bins=5, plot_types=None):
    """
    Mostra estadístiques i gràfics per columnes numèriques.
    Paràmetres:
        plot_types: llista amb tipus de gràfic a mostrar. Ex: ['hist', 'count', 'bar', 'box', 'violin']
    """
    import pandas as pd
    import numpy as np
    import matplotlib.pyplot as plt
    import seaborn as sns

    if col not in df.columns:
        raise ValueError(f"Columna no trobada: {col}")

    n = len(df)
    vc = df[col].value_counts(dropna=False)
    pct = (vc / n * 100).round(2)
    summary_df = pd.DataFrame({'count': vc, 'pct': pct})

    print(f"\nColumn: '{col}' — {n} files")
    print(summary_df)

    # Afegim summary sobre el target
    if target in df.columns:
        print(f"\nDistribució de '{target}' per valors únics de '{col}':")
        target_summary = df.groupby(col)[target].value_counts(dropna=False).unstack(fill_value=0)
        print(target_summary)

    if plot_types is None:
        plot_types = ['hist', 'box']

    if show_plots:
        try:
            sns.set(style="whitegrid")
            if 'hist' in plot_types:
                plt.figure(figsize=(6,3))
                sns.histplot(data=df, x=col, bins=bins, kde=True, color='C0')
                plt.title(f'Distribució de {col}')
                plt.xlabel(col)
                plt.tight_layout()
                plt.show()
            if 'count' in plot_types:
                plt.figure(figsize=(6,3))
                sns.countplot(x=col, data=df, color='C2')
                plt.title(f'Recompte de valors únics per {col}')
                plt.xlabel(col)
                plt.tight_layout()
                plt.show()
            if 'bar' in plot_types and target in df.columns:
                sub = df[[col, target]].dropna()
                # Si bins és None, utilitza els valors únics com a "bins"
                if bins is None:
                    bin_labels = sorted(sub[col].unique())
                    surv_per_bin = sub.groupby(col)[target].sum().reindex(bin_labels)
                    plt.figure(figsize=(7,3))
                    surv_per_bin.plot(kind='bar', color='C1')
                    plt.title(f'Nombre de {target} per valor únic de {col}')
                    plt.ylabel(f'Count de {target}')
                    plt.xlabel(f'{col} (valors únics)')
                    plt.tight_layout()
                    plt.show()
                else:
                    binned = pd.cut(sub[col], bins=bins)
                    surv_per_bin = sub.groupby(binned)[target].sum()
                    plt.figure(figsize=(7,3))
                    surv_per_bin.plot(kind='bar', color='C1')
                    plt.title(f'Nombre de {target} per bins de {col}')
                    plt.ylabel(f'Count de {target}')
                    plt.xlabel(f'{col} (bins)')
                    plt.tight_layout()
                    plt.show()
        except Exception:
            pass


#### GAME ID
Se suposa que el id de la partida ha de ser únic, però per si de cas anem a comprovar-ho

In [54]:
# Comporvar que el game id sigui únic

unique_game_ids = DataSet_cleaned['gameId'].nunique()
total_game_ids = len(DataSet_cleaned['gameId'])
print(f"Nombre únic de gameId: {unique_game_ids} de {total_game_ids} files.")
if unique_game_ids == total_game_ids:
    print("El gameId és únic per a cada fila.")
else:
    print("El gameId NO és únic per a cada fila.")

Nombre únic de gameId: 51053 de 51053 files.
El gameId és únic per a cada fila.


#### creationTime
De primeres no crec que sigui molt rellevant pel desenvolupament de l'anàlisi, però anem a veure

In [55]:
# Comprovar el nombre de valors únics (in Epoch format)

numeric_cols = DataSet_cleaned['creationTime'].nunique()
print(f"Nombre únic de creationTime: {numeric_cols} de {len(DataSet_cleaned)} files.")

Nombre únic de creationTime: 51052 de 51053 files.


Observem que hi ha dos partides diferents que és van crear en el mateix temps. Per aprofundir una mica més en aquests dos casos, mirem quin dels dos equips va guanyar.

In [56]:
# Comprovar guanyador de les partides que tenen el mateix creation time
duplicate_creation_times = DataSet_cleaned[DataSet_cleaned.duplicated(subset=['creationTime'], keep=False)]
print(duplicate_creation_times[['gameId', 'creationTime', 'winner']])


           gameId   creationTime  winner
34308  3301531504  1502619256635       2
43811  3301502454  1502619256635       1


Bé són partides diferents, però les han guanyat equips diferents. En aquest cas no aniré més enllà, ja que no li veig cap altra relació que pugui haver entre *winner i creationTime*. Encara que no deixo la probabilitat que tinguin altres camps en comú, però ja seran mirats a continuació

#### gameDuration
A continuació el que veig més seguir és mirar si hi ha alguna partida que tingui el mateix temps i si hi ha mirar l'equip guanyat. També mirar la mitjana de temps de les partides, per tenir una idea.

In [57]:
# Mirar si hi ha dos partides amb la mateixa durada i el guanyador de cadascuna
duplicate_durations = DataSet_cleaned[DataSet_cleaned.duplicated(subset=['gameDuration'], keep=False)]


if not duplicate_durations.empty:
    print(f"Hi ha {len(duplicate_durations)} partides amb la mateixa durada.")
else:
    print("No hi ha partides amb la mateixa durada.")

# Agrupar per temps de durada i mostrar un recompte dels guanyadors per a cada durada duplicada
grouped_duplicates = duplicate_durations.groupby('gameDuration')
for duration, group in grouped_duplicates:
    print(f"\nDurada de la partida: {duration} segons")
 
    # Comptabilitzar i el percentatge de guanyadors en relació al nombre de partides amb aquesta durada
    print(group['winner'].value_counts(normalize=True) * 100)
    print(group['winner'].value_counts())



Hi ha 50740 partides amb la mateixa durada.



Durada de la partida: 190 segons
winner
1    50.0
2    50.0
Name: proportion, dtype: float64
winner
1    1
2    1
Name: count, dtype: int64

Durada de la partida: 191 segons
winner
2    54.545455
1    45.454545
Name: proportion, dtype: float64
winner
2    6
1    5
Name: count, dtype: int64

Durada de la partida: 192 segons
winner
2    55.813953
1    44.186047
Name: proportion, dtype: float64
winner
2    24
1    19
Name: count, dtype: int64

Durada de la partida: 193 segons
winner
2    56.578947
1    43.421053
Name: proportion, dtype: float64
winner
2    43
1    33
Name: count, dtype: int64

Durada de la partida: 194 segons
winner
2    51.960784
1    48.039216
Name: proportion, dtype: float64
winner
2    53
1    49
Name: count, dtype: int64

Durada de la partida: 195 segons
winner
1    55.670103
2    44.329897
Name: proportion, dtype: float64
winner
1    54
2    43
Name: count, dtype: int64

Durada de la partida: 196 segons
winner
1    56.692913
2    43.307087
Name: proportion, dtype:

#### seasonID 
Mirem la quantitat de valors únics que tenim

In [58]:
# Mirar quantitat de valors únics de la columna

numeric_cols = DataSet_cleaned['seasonId'].nunique()
unic_vals = DataSet_cleaned['seasonId'].unique()

print(f"Nombre únic de seasonId: {numeric_cols} de {len(DataSet_cleaned)} files.")
print(f"Valors únics de seasonId: {unic_vals}")

Nombre únic de seasonId: 1 de 51053 files.
Valors únics de seasonId: [9]


#### Winner
Mirar només nombre i percentatge de partides gunyades de cadascun dels equips 

In [61]:
# Nombre i percentatge de partides guanyades de cada equip

winner_count = DataSet_cleaned['winner'].value_counts()
winner_percentage = (winner_count / len(DataSet_cleaned) * 100).round(2)

print("Nombre de partides guanyades per equip:")
print(winner_count)
print("\nPercentatge de partides guanyades per equip:")
print(winner_percentage)

Nombre de partides guanyades per equip:
winner
1    25857
2    25196
Name: count, dtype: int64

Percentatge de partides guanyades per equip:
winner
1    50.65
2    49.35
Name: count, dtype: float64


#### T1
Anem a separar 


#### Corelacions entre les columnes
